In [5]:
from speak_note.tools.llms import llm_list
from speak_note.tools.context import retriever_configs, context_configs, Context
from speak_note.work_flows import basic_RAG
from speak_note.tools.myPDFparser import upstageParser2Document
import speak_note.prompts.prompt as  prompt

import asyncio

# 서버 첫 INIT
### 해야할일
1. 문서pdf -> 문서json
2. 메타데이터에 {문서요약 : 문서json경로} append하기.

## 문서pdf -> 문서json

In [2]:
from speak_note.engine.init.load_docs import load_docs

result = await load_docs()

[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/[별표2] 사용료 및 수강료 반환기준(제14조 관련).pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/부산광역시 금정구 도서관 운영 규정.pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/[별표1] 사용료 기준(제12조제2항 관련).pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/sample_AI_Brief.pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/부산광역시 금정구 독서문화 진흥 조례.pdf
[PARSER] parsing 완료: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/부산광역시 금정구 독서문화 진흥 조례.pdf / 소요시간: 1.52초
[PARSER] parsing 완료: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/[별표2] 사용료 및 수강료 반환기준(제14조 관련).pdf / 소요시간: 1.64초
[PARSER] parsing 완료: /Users/yujin/Desk

#### 실전 코드

In [97]:
import os
from pathlib import Path
import asyncio
import json

data_root_str = "/Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas"
rawDocs_name = "rawDocs"
parsed_docs_name = "parsedDos"

data_root = Path(data_root_str)
raw_docs_root = data_root.joinpath(rawDocs_name)
parsed_docs_root = data_root.joinpath(parsed_docs_name)

# 원본 파일 리스트 (파일명, Path)
files = [(f.name, f) for f in raw_docs_root.rglob("*.pdf") if f.is_file()]

# 대응되는 JSON 저장 경로 생성
def pdf2json_path(pdf_file: str):
    return parsed_docs_root.joinpath(Path(pdf_file).stem + ".json")

save_names = [pdf2json_path(f[0]) for f in files]

# 비동기 병렬 파싱
tasks = [asyncio.create_task(upstageParser2Document(str(f[1]))) for f in files]
results = await asyncio.gather(*tasks)
# results: [ [Document(page1), Document(page2), ...], [Document(page1), ...], ... ]


[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/[별표2] 사용료 및 수강료 반환기준(제14조 관련).pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/부산광역시 금정구 도서관 운영 규정.pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/[별표1] 사용료 기준(제12조제2항 관련).pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/sample_AI_Brief.pdf
[PARSER] parsing 시작: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/부산광역시 금정구 독서문화 진흥 조례.pdf
[PARSER] parsing 완료: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/[별표1] 사용료 기준(제12조제2항 관련).pdf / 소요시간: 1.54초
[PARSER] parsing 완료: /Users/yujin/Desktop/코딩shit/python_projects/대한민국해커톤/민원마루ver1/datas/rawDocs/[별표2] 사용료 및 수강료 반환기준(제14조 관련).pdf / 소요시간: 1.64초
[PARSER] parsin

In [102]:
# 파일별로 저장
for i, save_name in enumerate(save_names):
    docs = results[i]  # 한 pdf의 여러 페이지 Document 리스트
    # Document → dict 변환
    docs_serialized = [d.dict() for d in docs]  

    with save_name.open("w", encoding="utf-8") as f:
        json.dump(docs_serialized, f, ensure_ascii=False, indent=2)


/var/folders/6q/q927ypwn4ll9d4rvjq9gl1qw0000gn/T/ipykernel_39869/192413637.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  docs_serialized = [d.dict() for d in docs]


### 다시 json파일 불러오기

In [108]:
from speak_note.tools.json_tool import load_json

docs = load_json(save_names[3])
docs

[Document(metadata={'page': 1}, page_content="1. 정책/법제\n2. 기업/산업\n3. 기술/연구\n4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표\nKEY Contents\n미국 바이든 대통령이 '안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령'에 서명하고\n광범위한 행정 조치를 명시\n■ 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자\n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\n바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\n■ 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과\n사용을 보장하기 위한 행정명령을 발표\n· 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호\n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄\n(AI 안전과 보안 기준) 강력한 AI 시스템을 개발하는 기업에게 안전 테스트 결과와 시스템에 관한\n주요 정보를 미국 정부와 공유할 것을 요구하고, AI 시스템의 안전성과 신뢰성 확인을 위한 표준 및\nAI 생성 콘텐츠 표시를 위한 표준과 모범사례 확립을 추진\n· △1026 플롭스(FLOPS, Floating Point Operation Per Second)를 초과하는 컴퓨팅 성능 또는 생물학적\n서열 데이터를 주로 사용하고 1023플롭스를 초과하는 컴퓨팅 성능을 사용하는 모델 △단일 데이터센터에서\n1,000Gbit/s 이상의 네트워킹으로 연결되며 AI 훈련에서 이론상 최대 1020 플롭스를 처리할 수 있는\n컴퓨팅 용량을 갖춘 컴퓨팅 클러스터가 정보공유 요구대상\n(형평성과 시민권 향상) 법률, 주택, 보건 분야에서 AI의 무책임한 사용으로 인한 차별과 편견 및 기타\n문제를 방지하는 조치를 확대\n● 형사사법 시스템에서 AI 사용 모

#### 요약 데이터 및 메타데이터 생성하기